In [1]:
import re
import string
import itertools as itools
import heapq as hq

class Maze:
    
    def __init__(self, dictionary):
        
        self.maze = dictionary
        self.adjacent = [1, -1, 1j, -1j]
        self.keys = {self.maze[x]: complex(x) for x in self.maze if self.maze[x] in string.ascii_lowercase}
        
        for pos, elem in self.maze.items():
            if elem == '@':
                self.entry = pos
                break
        
        return

    # Wrapper to access a given coordinate location. Returns 'None' if location is out of bound
    def access_loc(self, coord):
        
        found = None
        try:
            found = self.maze[coord]
        
        except:
            pass
        
        return found
        
    # Returns list of neighbours of given point in the form [[coordinates], [items]], 
    # where 'coordinates' and 'items' are in 1-to-1 correspondence.
    # kwarg 'select' is a regular expression that can be used to limit the search to
    # a given type of object. If not specified, all neighbors are returned.
    def near_neighbors(self, point, select=None):
        
        neighbor_list = [[], []]
        world = string.ascii_letters+'.'+'#'+'@'
        
        for adj in self.adjacent:
            
            neighbor = point+adj
            obj = self.access_loc(neighbor)
            if obj is not None:
                
                if select is None:
                    neighbor_list[0].append(neighbor)
                    neighbor_list[1].append(obj)
                    
                elif obj in re.findall(select, world):
                    neighbor_list[0].append(neighbor)
                    neighbor_list[1].append(obj)

                        
        return neighbor_list
    
    
    # This is not necessary, just a funny old thing that could be useful.
    # Fills all the dead ends of the maze with '#', effectively creating a new maze
    # which does not have dead ends. Could potentially solve
    def fill_dead_ends(self):
        
        complete = False
        
        while complete != True:
            
            complete = True
            
            for point in self.maze:
                
                if self.maze[point] != '.':
                    if self.maze[point] not in string.ascii_uppercase:
                        continue
                
                if self.near_neighbors(point)[1].count('#') >= 3:
                    self.maze[point] = '#'
                    complete = False
                    
        return
    
    
    def stringify(self):
        
        maze_width = int(max([x for x in self.maze if isinstance(x, int)]))
        y = 0
        maze_str = ''
        
        for n in range(len(self.maze)):

            x = n%(maze_width+1)
            maze_str += self.maze[x+y]
    
            if x == maze_width:
                y -= 1j
                x = 0
                maze_str += '\n'
    
            
        return maze_str
    
    
    # Calculates the (shortest) distance between any two keys in the maze, and also returns the
    # list of keys and doors encountered on the path from one to the other
    def distance(self, a, b):
        
        if a == b:
            return 0, []
                
        pos_a = self.entry if a == '@' else self.keys[a]
        pos_b = self.entry if b == '@' else self.keys[b]
        
        # Initialize list of heads, visited positions and found keys
        heads = [pos_a]
        found = {pos_a: a} 
        visited = [pos_a]
        d = 0
        
        while heads != []:
            
            d +=1            
            new_heads = []

            for head in heads:
                                                        #'+keys.upper()+found[head].upper()+'
                accessible = self.near_neighbors(head, select='[a-zA-Z@\.]')
                for i, x in enumerate(accessible[0]):
                    if x not in visited:
                        found[x] = found[head]
                        
                        if accessible[1][i] in string.ascii_letters:
                            found[x]+=accessible[1][i]
                        
                        new_heads.append(x)
                        visited.append(x)
                             
                del found[head]
            
            heads = new_heads[:]
            
            if pos_b in heads:
                break
            
        
        return d, found[pos_b]
        
        
    # Modified Dijkstra algorithm
    def dij_solver(self, rmap):
    
        # Unique identifier for each entry of the heap, in case entries have equal lengths
        unique = itools.count()
        # Initialize the heap as a list of tuples with (distance, uniqueID, current node, keys collected)
        heap = [(0, next(unique), '@', set())]
        # Stores the visited nodes
        G = {}
        #prev = {}
        
        while heap:
            
            # Pop head from heap, the node with current minimum distance
            curr_dist, dump, node, keys = hq.heappop(heap)
            
            # Success statement
            if keys >= set(self.keys):
                return curr_dist, node
            
            # Choose a new possible node from the missing ones
            for new_node in set(self.keys)-set(keys):
                
                # Copy the keys of the node
                new_keys = set(keys)
                # Lookup to the map to see distance between current node and new node
                d, found = rmap[frozenset((node, new_node))]
                # Doors on path
                doors = set([x.lower() for x in found if x.isupper()])
                # Keys on path + old keys. 
                new_keys.update([x for x in found if x.islower()])

                # If we have all the necessary keys to go to the new node proceed.
                # It is important to update the keys since in a case like
                # c-----a---A----i
                # and going from 'c' to 'i' (with no other keys, for example) 
                # we find door A but also its relative key 'a'.
                # If we do not update the keys with the one found on the path the condition below
                # will not be satisfied even though the path is in fact traversable.
                if new_keys >= doors:
                
                    # See if the pair (new node, {keys collected}) is already in the graph G.
                    # If it is, apply Dijkstra condition to the node and push to the heap.
                    # If not, add pair to the graph G and again push to the heap
                    try:
                        old_dist = G[(new_node, frozenset(new_keys))]
                        if old_dist > (curr_dist + d):
                            G[(new_node, frozenset(new_keys))] = curr_dist + d
                            #prev[new_node] = node
                            hq.heappush(heap, (curr_dist+d, next(unique), new_node, new_keys))
                                    
                    except:
                        G[(new_node, frozenset(new_keys))] = curr_dist + d
                        #prev[new_node] = node
                        hq.heappush(heap, (curr_dist+d, next(unique), new_node, new_keys))
                        
                        
        return 

In [2]:
x = 0
y = 0
m = dict()

with open('input.txt', 'r') as infile:
    for line in infile:
        symb = list(line.replace('\n', ''))
        for s in symb:
            m[x+y] = s
            x +=1
        x = 0 
        y -= 1j
        
# Initialize Maze object
maze = Maze(m)
# Fill dead ends
maze.fill_dead_ends()
# Uncomment to see the new graph with no dead ends 
#print(maze.stringify())

# Create a map which for every pair of keys stores the distance and the
# keys+doors found on the path that connects them
roadmap = {}
for u, v in itools.combinations(''.join(maze.keys)+'@', 2):
    d, found = maze.distance(u, v)
    roadmap[frozenset((u, v))] = (d, set(found))

print(roadmap)

{frozenset({'u', 'z'}): (238, {'u', 'z', 'Q'}), frozenset({'z', 'b'}): (214, {'z', 'Q', 'b'}), frozenset({'z', 'l'}): (156, {'v', 'z', 'l'}), frozenset({'m', 'z'}): (440, {'u', 'A', 'f', 'T', 'K', 'm', 'z', 'R', 'w', 'j', 'Q', 'F', 'J', 'H'}), frozenset({'z', 'w'}): (272, {'u', 'A', 'z', 'w', 'Q'}), frozenset({'z', 'e'}): (232, {'v', 'z', 'e', 'C'}), frozenset({'z', 'i'}): (522, {'T', 'm', 'z', 'R', 'u', 'Y', 'K', 'o', 'j', 'Q', 'f', 'i', 'O', 'M', 'J', 'H', 'A', 'w', 'F', 'X', 'x'}), frozenset({'z', 'y'}): (454, {'u', 'A', 'f', 'T', 'y', 'K', 'm', 'z', 'R', 'w', 'j', 'Q', 'F', 'M', 'J', 'H'}), frozenset({'z', 'x'}): (478, {'u', 'A', 'f', 'T', 'Y', 'K', 'm', 'z', 'R', 'w', 'j', 'Q', 'F', 'M', 'J', 'H', 'x'}), frozenset({'v', 'z'}): (38, {'v', 'z'}), frozenset({'f', 'z'}): (414, {'u', 'A', 'f', 'T', 'K', 'z', 'R', 'w', 'j', 'Q', 'J', 'H'}), frozenset({'z', 'o'}): (494, {'u', 'A', 'f', 'T', 'Y', 'K', 'm', 'z', 'R', 'o', 'w', 'j', 'Q', 'F', 'X', 'M', 'J', 'H', 'x'}), frozenset({'z', 'k'})

In [27]:
# Call Dijkstra algorithm to solve the maze
min_dist, last = maze.dij_solver(roadmap)
print(min_dist, last)

136 m
